In [1]:
# code from https://www.youtube.com/watch?v=m1vctOrYrTY&t=1566s

import sys
import math
import glob
import os
import numpy as np
import cv2
import queue
import random 
import time

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

def processImage(image):
    image = np.array(image.raw_data)
    img = image.reshape((600,800,4))
    img = img[:,:,:3]

    cv2.imshow('img',img)
    cv2.waitKey(0)

def get_speed(vehicle):
    vel = vehicle.get_velocity()
    return 3.6*math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)


In [2]:
actor_list = []

client = carla.Client('localhost',2000)
client.set_timeout(5.0)
world = client.get_world()
#map = world.get_map()

blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle'))
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(bp, transform)
actor_list.append(vehicle)
print('created %s' % vehicle.type_id)

# Let's put the vehicle to drive around.
vehicle.set_autopilot(True)

created vehicle.audi.etron


In [3]:
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','800')
camera_bp.set_attribute('image_size_y','600')
camera_bp.set_attribute('fov','90')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
actor_list.append(camera)
print('created %s' % camera.type_id)


created sensor.camera.rgb


In [4]:
camera.listen(lambda image: processImage(image))


In [ ]:
print('destroying actors')
camera.destroy()
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])
print('done.')